<h1 style="text-align:center">Deep Learning   </h1>
<h1 style="text-align:center"> Lab Session 2 - 1.5 Hours </h1>
<h1 style="text-align:center"> Convolutional Neural Network (CNN) for Handwritten Digits Recognition</h1>

<b> Group name:</b> DeepLearning19 - Deodato Giacomo - Bucquet Anthime
 
 
The aim of this session is to practice with Convolutional Neural Networks. Each group should fill and run appropriate notebook cells. 


Generate your final report (export as HTML) and upload it on the submission website http://bigfoot-m1.eurecom.fr/teachingsub/login (using your deeplearnXX/password). Do not forget to run all your cells before generating your final report and do not forget to include the names of all participants in the group. The lab session should be completed and submitted by May 30th 2018 (23:59:59 CET).

# Introduction

In the previous Lab Session, you built a Multilayer Perceptron for recognizing hand-written digits from the MNIST data-set. The best achieved accuracy on testing data was about 97%. Can you do better than these results using a deep CNN ?
In this Lab Session, you will build, train and optimize in TensorFlow one of the early Convolutional Neural Networks,  **LeNet-5**, to go to more than 99% of accuracy. 






# Load MNIST Data in TensorFlow
Run the cell below to load the MNIST data that comes with TensorFlow. You will use this data in **Section 1** and **Section 2**.

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
X_train, y_train = mnist.train.images, mnist.train.labels
X_validation, y_validation = mnist.validation.images, mnist.validation.labels
X_test, y_test = mnist.test.images, mnist.test.labels

print("Image Shape: {}".format(X_train[0].shape))
print("Training Set: {} samples".format(len(X_train)))
print("Validation Set: {} samples".format(len(X_validation)))
print("Test Set: {} samples".format(len(X_test)))

epsilon = 1e-10 # this is a parameter you will use later

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Image Shape: (784,)
Training Set: 55000 samples
Validation Set: 5000 samples
Test Set: 10000 samples


# Section 1 : My First Model in TensorFlow

Before starting with CNN, let's train and test in TensorFlow the example
**y=softmax(Wx+b)** seen in the first lab. 

This model reaches an accuracy of about 92 %.
You will also learn how to launch the TensorBoard https://www.tensorflow.org/get_started/summaries_and_tensorboard to visualize the computation graph, statistics and learning curves. 

<b> Part 1 </b> : Read carefully the code in the cell below. Run it to perform training. 

In [2]:
#STEP 1

# Parameters
learning_rate = 0.01
training_epochs = 40
batch_size = 128
display_step = 1
logs_path = 'log_files/'  # useful for tensorboard

# tf Graph Input:  mnist data image of shape 28*28=784
x = tf.placeholder(tf.float32, [None, 784], name='InputData')
# 0-9 digits recognition,  10 classes
y = tf.placeholder(tf.float32, [None, 10], name='LabelData')

# Set model weights
W = tf.Variable(tf.zeros([784, 10]), name='Weights')
b = tf.Variable(tf.zeros([10]), name='Bias')

# Construct model and encapsulating all ops into scopes, making Tensorboard's Graph visualization more convenient
with tf.name_scope('Model'):
    # Model
    pred = tf.nn.softmax(tf.matmul(x, W) + b) # Softmax
with tf.name_scope('Loss'):
    # Minimize error using cross entropy
    # We use tf.clip_by_value to avoid having too low numbers in the log function
    cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(tf.clip_by_value(pred, epsilon, 1.0)), reduction_indices=1))
with tf.name_scope('SGD'):
    # Gradient Descent
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
with tf.name_scope('Accuracy'):
    # Accuracy
    acc = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    acc = tf.reduce_mean(tf.cast(acc, tf.float32))
    
# Initializing the variables
init = tf.global_variables_initializer()
# Create a summary to monitor cost tensor
tf.summary.scalar("Loss", cost)
# Create a summary to monitor accuracy tensor
tf.summary.scalar("Accuracy", acc)
# Merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()

#STEP 2 

# Launch the graph for training
with tf.Session() as sess:
    sess.run(init)
    # op to write logs to Tensorboard
    summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size, shuffle=(i==0))
            # Run optimization op (backprop), cost op (to get loss value)
            # and summary nodes
            _, c, summary = sess.run([optimizer, cost, merged_summary_op],
                                     feed_dict={x: batch_xs, y: batch_ys})
            # Write logs at every iteration
            summary_writer.add_summary(summary, epoch * total_batch + i)
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if (epoch+1) % display_step == 0:
            print("Epoch: ", '%02d' % (epoch+1), "  =====> Loss=", "{:.9f}".format(avg_cost))

    print("Optimization Finished!")
    summary_writer.flush()

    # Test model
    # Calculate accuracy
    print("Accuracy:", acc.eval({x: mnist.test.images, y: mnist.test.labels}))

Epoch:  01   =====> Loss= 1.289101143
Epoch:  02   =====> Loss= 0.732894713
Epoch:  03   =====> Loss= 0.600314501
Epoch:  04   =====> Loss= 0.536638115
Epoch:  05   =====> Loss= 0.497763241
Epoch:  06   =====> Loss= 0.471036514
Epoch:  07   =====> Loss= 0.451162340
Epoch:  08   =====> Loss= 0.435974350
Epoch:  09   =====> Loss= 0.423423332
Epoch:  10   =====> Loss= 0.413314678
Epoch:  11   =====> Loss= 0.404588916
Epoch:  12   =====> Loss= 0.396654918
Epoch:  13   =====> Loss= 0.390362654
Epoch:  14   =====> Loss= 0.384643417
Epoch:  15   =====> Loss= 0.379151668
Epoch:  16   =====> Loss= 0.374498023
Epoch:  17   =====> Loss= 0.370150838
Epoch:  18   =====> Loss= 0.366568124
Epoch:  19   =====> Loss= 0.362809329
Epoch:  20   =====> Loss= 0.359685079
Epoch:  21   =====> Loss= 0.356761633
Epoch:  22   =====> Loss= 0.353807260
Epoch:  23   =====> Loss= 0.351510445
Epoch:  24   =====> Loss= 0.348832101
Epoch:  25   =====> Loss= 0.346561326
Epoch:  26   =====> Loss= 0.344264160
Epoch:  27  

<b> Part 2  </b>: Using Tensorboard, we can  now visualize the created graph, giving you an overview of your architecture and how all of the major components  are connected. You can also see and analyse the learning curves. 

To launch tensorBoard: 
- Open a Terminal and run the command line **"tensorboard --logdir=lab_2/log_files/"**
- Click on "Tensorboard web interface" in Zoe  


Enjoy It !! 


# Section 2 : The 99% MNIST Challenge !

<b> Part 1 </b> : LeNet5 implementation

You are now more familar with **TensorFlow** and **TensorBoard**. In this section, you are to build, train and test the baseline [LeNet-5](http://yann.lecun.com/exdb/lenet/)  model for the MNIST digits recognition problem.  

Then, you will make some optimizations to get more than 99% of accuracy.

For more informations, have a look at this list of results: http://rodrigob.github.io/are_we_there_yet/build/classification_datasets_results.html


<img src="lenet.png",width="800" height="600" align="center">
<center><span>Figure 1: Lenet-5 </span></center>





The LeNet architecture takes a 28x28xC image as input, where C is the number of color channels. Since MNIST images are grayscale, C is 1 in this case.

--------------------------
**Layer 1 - Convolution (5x5):** The output shape should be 28x28x6. **Activation:** ReLU. **MaxPooling:** The output shape should be 14x14x6.

**Layer 2 - Convolution (5x5):** The output shape should be 10x10x16. **Activation:** ReLU. **MaxPooling:** The output shape should be 5x5x16.

**Flatten:** Flatten the output shape of the final pooling layer such that it's 1D instead of 3D.  You may need to use tf.reshape.

**Layer 3 - Fully Connected:** This should have 120 outputs. **Activation:** ReLU.

**Layer 4 - Fully Connected:** This should have 84 outputs. **Activation:** ReLU.

**Layer 5 - Fully Connected:** This should have 10 outputs. **Activation:** softmax.


<b> Question 2.1.1 </b>  Implement the Neural Network architecture described above.
For that, your will use classes and functions from  https://www.tensorflow.org/api_docs/python/tf/nn. 

We give you some helper functions for weigths and bias initilization. Also you can refer to section 1. 


In [2]:
# Functions for weigths and bias initilization 
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.05, shape=[shape])
    return tf.Variable(initial)

In [3]:
def LeNet5_Model(image):    
    x_image = tf.reshape(image, [-1, 28, 28, 1])
    # Convolutional Layer #1 - return a 28 X 28 X 6
    shape = [5, 5, 1, 6]
    conv1 = tf.nn.conv2d(input=x_image,
                         filter=weight_variable(shape),
                         strides=[1, 1, 1, 1],
                         padding='SAME') + bias_variable(shape[-1])

    conv1 = tf.nn.relu(conv1)

    # Pooling Layer #1
    pool1 = tf.nn.max_pool(value=conv1,
                               ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1],
                               padding='VALID')

    print(pool1)

    # Convolutional Layer #2 and Pooling Layer #2
    shape = [5, 5, 6, 16]

    conv2 = tf.nn.conv2d(input=pool1,
                         filter=weight_variable(shape),
                         strides=[1, 1, 1, 1],
                         padding='VALID') + bias_variable(shape[-1])

    conv2 = tf.nn.relu(conv2)

    print(conv2)
    # Pooling Layer #1
    pool2 = tf.nn.max_pool(value=conv2,
                               ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1],
                               padding='VALID')
    print(pool2)

    # Flatten Layer

    flatten_layer = tf.contrib.layers.flatten(pool2)

    print(flatten_layer)


    #Fully-connected Layer 1
    shape = [400,120]
    fc_layer1 = tf.matmul(flatten_layer, weight_variable(shape)) + bias_variable(shape[-1])
    fc_layer1=tf.nn.relu(fc_layer1)
    print(fc_layer1)

    #Fully-connected Layer 2
    shape = [120,84]
    fc_layer2 = tf.matmul(fc_layer1, weight_variable(shape)) + bias_variable(shape[-1])
    fc_layer2= tf.nn.relu(fc_layer2)
    print(fc_layer2)

    #Fully-connected Layer 3
    shape = [84,10]
    fc_layer3 = tf.matmul(fc_layer2, weight_variable(shape)) + bias_variable(shape[-1])
    fc_layer3 = tf.nn.softmax(fc_layer3)
    print(fc_layer3)
    return fc_layer3

In [11]:
model = LeNet5_Model(X_train)

Tensor("MaxPool:0", shape=(55000, 14, 14, 6), dtype=float32)
Tensor("Relu_1:0", shape=(55000, 10, 10, 16), dtype=float32)
Tensor("MaxPool_1:0", shape=(55000, 5, 5, 16), dtype=float32)
Tensor("Flatten/flatten/Reshape:0", shape=(55000, 400), dtype=float32)
Tensor("Relu_2:0", shape=(55000, 120), dtype=float32)
Tensor("Relu_3:0", shape=(55000, 84), dtype=float32)
Tensor("Softmax:0", shape=(55000, 10), dtype=float32)


<b> Question 2.1.2. </b>  Calculate the number of parameters of this model 

 Your answer goes here in details 

<b> Question 2.1.3. </b>  Define your model, its accuracy and the loss function according to the following parameters (you can look at Section 1 to see what is expected):

     Learning rate: 0.001
     Loss Fucntion: Cross-entropy
     Optimizer: tf.train.GradientDescentOptimizer
     Number of epochs: 40
     Batch size: 128

In [13]:
tf.reset_default_graph() # reset the default graph before defining a new model


x = tf.placeholder(tf.float32, shape=[None, 784], name='x')
y = tf.placeholder(tf.float32, [None, 10], name='predict_label')

model = LeNet5_Model(x)
# Parameters
learning_rate = 0.1
training_epochs = 40
batch_size = 128
display_step = 10
logs_path = 'log_files/'  # useful for tensorboard
saving_path = 'models/' #to keep a copy of te model


with tf.name_scope('Loss'):
    # Minimize error using cross entropy
    cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(model), reduction_indices=1))
with tf.name_scope('SGD'):
    # Gradient Descent
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)


Tensor("MaxPool:0", shape=(?, 14, 14, 6), dtype=float32)
Tensor("Relu_1:0", shape=(?, 10, 10, 16), dtype=float32)
Tensor("MaxPool_1:0", shape=(?, 5, 5, 16), dtype=float32)
Tensor("Flatten/flatten/Reshape:0", shape=(?, 400), dtype=float32)
Tensor("Relu_2:0", shape=(?, 120), dtype=float32)
Tensor("Relu_3:0", shape=(?, 84), dtype=float32)
Tensor("Softmax:0", shape=(?, 10), dtype=float32)


<b> Question 2.1.4. </b>  Implement the evaluation function for accuracy computation 

In [14]:
def evaluate(logits, labels):
    acc = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
    accu = tf.reduce_mean(tf.cast(acc, tf.float32))
    return accu

In [15]:
with tf.name_scope('Accuracy'):
    accu = evaluate(model, y)

<b> Question 2.1.5. </b>  Implement training pipeline and run the training data through it to train the model.

- Before each epoch, shuffle the training set. 
- Print the loss per mini batch and the training/validation accuracy per epoch. (Display results every 100 epochs)
- Save the model after training
- Print after training the final testing accuracy 



In [16]:
# Initializing the variables
init = tf.global_variables_initializer()
# Create a summary to monitor cost tensor
tf.summary.scalar("Loss_LeNet-5_SGD", cost)
# Create a summary to monitor accuracy tensor
tf.summary.scalar("Accuracy_LeNet-5_SGD", accu)
# Merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()

In [17]:
# Initializing the variables
def train(training_epochs, batch_size, cost, optimizer, acc):
    with tf.Session() as sess:
        sess.run(init)
        # op to write logs to Tensorboard
        summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
        # Training cycle
        for epoch in range(training_epochs):
            avg_cost = 0.
            total_batch = int(mnist.train.num_examples/batch_size)
            # Loop over all batches
            for i in range(total_batch):
                batch_xs, batch_ys = mnist.train.next_batch(batch_size, shuffle=(i==0))
                # Run optimization op (backprop), cost op (to get loss value)
                # and summary nodes
                _, c, summary = sess.run([optimizer, cost, merged_summary_op],
                                         feed_dict={x: batch_xs, y: batch_ys})
                # Write logs at every iteration
                summary_writer.add_summary(summary, epoch * total_batch + i)
                # Compute average loss
                avg_cost += c / total_batch
            # Display logs per epoch step
            if (epoch+1) % display_step == 0:
                print("Epoch: ", '%02d' % (epoch+1), "  =====> Loss=", "{:.9f}".format(avg_cost))
                print("Accuracy = ", accu.eval(feed_dict={x: mnist.test.images, y: mnist.test.labels}))
        print("Optimization Finished!")

        # Test model
        # Calculate accuracy
        print("Final Accuracy = ", accu.eval(feed_dict={x: mnist.test.images, y: mnist.test.labels}))
        summary_writer.flush()


In [18]:
train(training_epochs, batch_size, cost, optimizer,accu)

Epoch:  10   =====> Loss= 0.022794240
Accuracy =  0.9862
Epoch:  20   =====> Loss= nan
Accuracy =  0.098
Epoch:  30   =====> Loss= nan
Accuracy =  0.098
Epoch:  40   =====> Loss= nan
Accuracy =  0.098
Optimization Finished!
Final Accuracy =  0.098


<b> Question 2.1.6 </b> : Use TensorBoard to visualise and save loss and accuracy curves. 
You will save figures in the folder **"lab_2/MNIST_figures"** and display them in your notebook.

Please put your loss and accuracy curves here.

<b> Part 2 </b> : LeNET 5 Optimization


<b> Question 2.2.1 </b>

- Retrain your network with AdamOptimizer and then fill the table above:


| Optimizer            |  Gradient Descent  |    AdamOptimizer    |
|----------------------|--------------------|---------------------|
| Testing Accuracy     |         ???        |        ???          |       
| Training Time        |         ???        |        ???          |  

- Which optimizer gives the best accuracy on test data?

**Your answer:** ...


In [ ]:
tf.reset_default_graph()
# your implementation goes here

<b> Question 2.2.2</b> Try to add dropout (keep_prob = 0.75) before the first fully connected layer. You will use tf.nn.dropout for that purpose. What accuracy do you achieve on testing data?

**Accuracy achieved on testing data:** ...

In [ ]:
def LeNet5_Model_Dropout(image):    
    # your implementation goes here

tf.reset_default_graph()
# your implementation goes here